<a href="https://colab.research.google.com/github/EmilyRousou/DSC511-Introduction/blob/main/Copy_of_DSC_511_Project_Combined1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DSC 511 Group Project

In [ ]:
# Installing pyspark
! pip3 install pyspark

In [ ]:
# Importing Libraries
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import seaborn as sns
from google.colab import drive
from pyspark.sql.functions import lower, regexp_replace
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import IDF
from pyspark.sql.functions import expr
from pyspark.ml.linalg import Vectors, DenseVector
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql.functions import avg, count, col

In [ ]:
spark = SparkSession.builder \
    .appName("Group_Project_Reviews_Covid") \
    .master("local") \
    .getOrCreate()

### Loading the datasets

In [ ]:
# Google drive and reading the csv of post dataset
drive.mount('/content/gdrive')
google_drive_path = "/content/gdrive/MyDrive/postcovid_reviews.csv"

Mounted at /content/gdrive


In [ ]:
# Google drive and reading the csv of pre dataset
drive.mount('/content/gdrive')
google_drive = "/content/gdrive/MyDrive/precovid_reviews.csv"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [21]:
post_df = spark.read.options(header='True', inferSchema='True', delimiter=',',multiline = True, escape = '"').csv(google_drive_path)
post_df.write.mode('overwrite').parquet('/tmp/my_data_parquet')
post_df = spark.read.parquet('/tmp/my_data_parquet')

In [22]:
post_df.show(10)

+--------------------+--------------------+--------------------+------+------------+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------+------+-----+----+--------------------+-------------------+
|         business_id|                name|             address|state_|        city|postal_code|     latitude|     longitude|stars|review_count|is_open|          categories|               hours|           review_id|             user_id|customer_stars|useful|funny|cool|               text_|              date_|
+--------------------+--------------------+--------------------+------+------------+-----------+-------------+--------------+-----+------------+-------+--------------------+--------------------+--------------------+--------------------+--------------+------+-----+----+--------------------+-------------------+
|2WRCcQATOe_Em0k61...|Yook Korean Grill...| 2408 Nanaimo Street|   

In [14]:
pre_df = spark.read.options(header='True', inferSchema='True', delimiter=',',multiline = True, escape = '"').csv(google_drive)
pre_df.write.mode('overwrite').parquet('/tmp/my_data_parquet')
pre_df = spark.read.parquet('/tmp/my_data_parquet')

### Data Preprocessing

Checking for missing values


In [15]:
# check for missing values
pre_df = pre_df.replace("NULL", None)
pre_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in pre_df.columns]).show()

+-----------+----+-------+------+----+-----------+--------+---------+-----+------------+-------+----------+------+---------+-------+--------------+------+-----+----+-----+-----+
|business_id|name|address|state_|city|postal_code|latitude|longitude|stars|review_count|is_open|categories| hours|review_id|user_id|customer_stars|useful|funny|cool|text_|date_|
+-----------+----+-------+------+----+-----------+--------+---------+-----+------------+-------+----------+------+---------+-------+--------------+------+-----+----+-----+-----+
|          0|   0|  13409|     0|   0|        384|       0|        0|    0|           0|      0|         0|216501|        0|      0|             0|     0|    0|   0|    1|    0|
+-----------+----+-------+------+----+-----------+--------+---------+-----+------------+-------+----------+------+---------+-------+--------------+------+-----+----+-----+-----+



In [23]:
# check for missing values
post_df = post_df.replace("NULL", None)
post_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in post_df.columns]).show()

+-----------+----+-------+------+----+-----------+--------+---------+-----+------------+-------+----------+-----+---------+-------+--------------+------+-----+----+-----+-----+
|business_id|name|address|state_|city|postal_code|latitude|longitude|stars|review_count|is_open|categories|hours|review_id|user_id|customer_stars|useful|funny|cool|text_|date_|
+-----------+----+-------+------+----+-----------+--------+---------+-----+------------+-------+----------+-----+---------+-------+--------------+------+-----+----+-----+-----+
|          0|   0|    992|     0|   0|          4|       0|        0|    0|           0|      0|         0|12232|        0|      0|             0|     0|    0|   0|    0|    0|
+-----------+----+-------+------+----+-----------+--------+---------+-----+------------+-------+----------+-----+---------+-------+--------------+------+-----+----+-----+-----+



Both datasets have some missing values in address,hours and postal code columns.
Since we will mostly focus on the reviews we decided to not drop them.
Only the one missing review was removed.

In [24]:
pre_df = pre_df.dropna(subset=["text_"])